In [1]:
import numpy as np
import pandas as pd
import os
import surprise
import pickle

In [2]:
MOVIE_LENS_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw', 'movie_lense'))
MOVIE_LENS_PATH

'/home/levin/escaping_echo_chambers/data/raw/movie_lense'

In [22]:
movielens = pd.read_csv(os.path.join(MOVIE_LENS_PATH, 'ratings.csv'))#.drop('timestamp', axis = 1)
movielens = movielens[movielens.timestamp > (max(movielens['timestamp']) - 10*365*24*60*60)]
movielens = movielens.drop('timestamp', axis = 1)

In [23]:
movielens.shape

(12287644, 3)

In [24]:
movielens = movielens.rename(columns = {'userId':'uid', 'movieId':'iid', 'rating':'rating'})

In [25]:
genome_scores = pd.read_csv(os.path.join(MOVIE_LENS_PATH, 'genome-scores.csv'))

In [26]:
#Let's look at how many movies have genome tags
movies_with_genome = genome_scores['movieId'].unique()
print('# of movies with genome tags:', len(movies_with_genome))

# of movies with genome tags: 13176


In [28]:
#Total number
print('# of movies in total:', len(movielens['iid'].unique()))

# of movies in total: 53811


In [30]:
movielens_short = movielens[movielens.iid.isin(movies_with_genome)]

In [31]:
#Well, this does not really make it shorter:(
movielens_short.shape, movielens.shape

((11978740, 3), (12287644, 3))

In [13]:
wide_mov = movielens_shorter.pivot(index='uid', columns='iid')
wide_mov.columns = ['iid_' + str(i) for i in wide_mov.columns.droplevel().rename(None)]

MemoryError: Unable to allocate 6.60 GiB for an array with shape (67269, 13176) and data type float64

In [72]:
np.__version__

'1.19.2'

In [69]:
movielens_short.groupby(['uid','iid'])['rating'].max().unstack()

ValueError: Unstacked DataFrame is too big, causing int32 overflow

In [37]:
movielens_shorter = movielens_short.sample(frac = 0.1, random_state=0)

In [38]:
movielens_shorter.shape

(2741736, 3)

In [39]:
movielens = movielens_shorter

In [40]:
lower_rating = movielens['rating'].min()
upper_rating = movielens['rating'].max()
print('Range:', lower_rating, upper_rating)

Range: 0.5 5.0


In [41]:
reader = surprise.Reader(rating_scale = (lower_rating, upper_rating))
data = surprise.Dataset.load_from_df(movielens, reader)

In [42]:
%%time
alg = surprise.SVDpp()
output = alg.fit(data.build_full_trainset())

CPU times: user 28min 7s, sys: 279 ms, total: 28min 7s
Wall time: 28min 8s


In [45]:
pred = alg.predict(uid = '50', iid = '52')
score = pred.est

In [46]:
score

3.53599671886717

In [55]:
data.df.shape

(2741736, 3)

In [56]:
alg

In [61]:
# filename = 'finalized_model.sav'
# pickle.dump(alg, open(filename, 'wb'))

In [59]:
loaded_model = pickle.load(open(filename, 'rb'))

In [60]:
loaded_model.predict(uid = '50', iid = '52').est

3.53599671886717

In [64]:
def recommend(uid, data, model):
    all_movie_ids = data.df['iid'].unique()
    uid_rated = data.df[data.df['uid'] == uid]['iid']
    movies_to_recommend = np.setdiff1d(all_movie_ids, uid_rated)
    prediction_set = [[uid, iid, 0] fo] #here 0 is arbitrary
    return len(movies_to_recommend), len(all_movie_ids) 

In [65]:
recommend(50, data, None)

(13173, 13174)

MemoryError: Unable to allocate 428. GiB for an array with shape (239590, 239590) and data type float64